In [23]:
import tensorflow as tf
print(tf.__version__)

1.5.1


# Tensorflow : multiplication
This just creates two matrices and multiplies them. Note that when I print out I do not get the expected answer. This is because tensorflow first creates an abstract version of the "graph." No computation happens until you create a session and run it. 

In [22]:
### Constants
x = tf.constant([5, 6])
y = tf.constant([6, 6])

### Multiply constants
out = tf.multiply(x,y)
print(out)

### Run model
with tf.Session() as sess:
    output = sess.run(out)
    print(output)

Tensor("Mul_7:0", shape=(2,), dtype=int32)
[30 36]


# Tensorflow : classification
Use mNist dataset to classify hand written digits

input > weight > hl (act) > weight > hl (act) > weight > out

cost = compare prediction to actual

optimizer = minimize cost (SGD, Adam, adagrad, ...)

Backpropogation = update the weights

feed forward + backprop = epoch

source : https://pythonprogramming.net/tensorflow-neural-network-session-machine-learning-tutorial/?completed=/tensorflow-deep-neural-network-machine-learning-tutorial/

In [73]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [54]:
mnist = input_data.read_data_sets("/Users/gloege/Documents/Projects/tmp", one_hot=True)

Extracting /Users/gloege/Documents/Projects/tmp/train-images-idx3-ubyte.gz
Extracting /Users/gloege/Documents/Projects/tmp/train-labels-idx1-ubyte.gz
Extracting /Users/gloege/Documents/Projects/tmp/t10k-images-idx3-ubyte.gz
Extracting /Users/gloege/Documents/Projects/tmp/t10k-labels-idx1-ubyte.gz


In [74]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    # OLD VERSION:
    #cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(prediction,y) )
    # NEW:
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        # OLD:
        #sess.run(tf.initialize_all_variables())
        # NEW:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 completed out of 10 loss: 1705332.88388
Epoch 1 completed out of 10 loss: 406657.497009
Epoch 2 completed out of 10 loss: 222337.056446
Epoch 3 completed out of 10 loss: 130912.199854
Epoch 4 completed out of 10 loss: 81054.4884989
Epoch 5 completed out of 10 loss: 53396.5511069


KeyboardInterrupt: 